# This is the 3rd installment of the Federal Open Data Analytics Edge series.

## Focus on Download of a GFY and process that.

### Link to previous Medium Article in the series - https://medium.com/@lulstrup/gaining-an-analytics-edge-using-federal-spending-open-data-b91b517f2c04

In [1]:
import os, glob, pathlib
import shutil
from datetime import datetime
from collections import Counter
import zipfile
import pandas as pd
import requests

In [ ]:
#TODO refactor code to use Python classes

In [ ]:
original_cwd = os.getcwd()
download_file_path = os.path.join(original_cwd, "USAspending_Archive_Downloads")
print(f"Path to working directory: {original_cwd}")
print(f"Path to USAspending Archive Zip file Dierectory: {download_file_path}")

## Use a manual lookup to get the date extension (can be automated with selenium package)

In [ ]:
url_GFY_archive = "https://www.usaspending.gov/download_center/award_data_archive"
print(f"Open this link in a new brower tab or window: {url_GFY_archive}")

### You stop at this point and just download files manually. For those who want something more automated, I've created the code below.

In [ ]:
def CurrentGFY():
    if datetime.now().month >= 10: 
        return datetime.now().year + 1
    else:
        return datetime.now().year
    
def Check_Archive_Filename_Format(filename_complete):
    base_filename = os.path.basename(filename_complete)
    # rule checks
    if not base_filename.startswith("FY"):
        return False
    if not base_filename.endswith(".zip") and not base_filename.endswith(".csv"):
        return False
    if " " in base_filename:
        return False
    if len(base_filename.split(".")) > 2:
        return False
    if "copy" in base_filename:
        return False
    
    return True

def Check_INPUT_FileName_Format_OK(full_download_filename):
    if full_download_filename == "START_PROCESS":
        return False
    full_download_filename = full_download_filename.strip() # removes leading and trailing spaces
    print(f"Filename you entered: {full_download_filename}")
    if not full_download_filename.endswith(".zip"):
        print("Error in filename format. Please try again...")
        return False
    if not full_download_filename.startswith("FY"):
        print("Error in filename format. Please try again...")
        return False
    print("Filename accepted.")
    print()
    return True

full_download_filename = "START_PROCESS"
assert Check_INPUT_FileName_Format_OK(full_download_filename) == False

In [ ]:
full_download_filename = "START_PROCESS"
while not Check_INPUT_FileName_Format_OK(full_download_filename):
    full_download_filename = input("Enter the FULL file name for the latest USAspending.gov archive file name (drag, copy, and paste): ")

In [ ]:
CurrentGFY()

In [ ]:
# USAspending Annual (GFY) Archive File Helper Functions

def Collect_GFY(filelist_complete_paths):
    list_of_GFY = []
    for filename_complete in existing_archive_filenames:
        filename = filename_complete.split("/")[-1]
        filename_GFY = filename[:6]
        list_of_GFY.append(filename_GFY)
    return dict(Counter(list_of_GFY))

#Collect_GFY(existing_archive_filenames)

def Get_Filename_Only(filename_to_check_complete_path):
    if os.path.isdir(filename_to_check_complete_path):
        #print(f"{filename_to_check_complete_path} is a directory. Ignore")
        return
    # assert TBD
    return os.path.basename(filename_to_check_complete_path)

def Get_GFY_from_file_path(filename_to_check_complete_path):
    if os.path.isdir(filename_to_check_complete_path):
        #print(f"{filename_to_check_complete_path} is a directory. Ignore")
        return
    filename_GFY = os.path.basename(filename_to_check_complete_path)[:6]
    assert filename_GFY[:2] == 'FY' # check this
    return filename_GFY

def Get_ArchiveDate_from_file_path(filename_to_check_complete_path):
    if os.path.isdir(filename_to_check_complete_path):
        #print(f"{filename_to_check_complete_path} is a directory. Ignore")
        return
    basefilename = os.path.basename(filename_to_check_complete_path)
    if filename_to_check_complete_path.endswith(".zip"):
        filename_latest_update = basefilename.split("_")[4][0:8]
    elif filename_to_check_complete_path.endswith(".csv"):
        filename_latest_update = basefilename.split("_")[4][0:8]
    else:
        filename_latest_update ="Filename_Format_Issue"
    # assert TBD
    return filename_latest_update

    
def Get_Info_On_Archive_Files(download_file_path):
    result = []
    existing_archive_filenames = sorted(glob.glob(os.path.join(download_file_path, "*"))) # display existing files in the folder
    for filename_complete in existing_archive_filenames:
        if Get_GFY_from_file_path(filename_complete): #ignore None when it encounters a directory
            record = {'GFY' : Get_GFY_from_file_path(filename_complete), 
                      'Archive_Date' : Get_ArchiveDate_from_file_path(filename_complete),
                      'Filename' : Get_Filename_Only(filename_complete),
                     }
            result.append(record.copy())
    df_archive_local_file_info = pd.DataFrame(result).sort_values(by='GFY')
    df_archive_local_file_info = df_archive_local_file_info[['GFY', 'Archive_Date', 'Filename']]
    return df_archive_local_file_info

# check if download directory exists, if not, create it
# if the directory exists, whoch files should be updated?
print("WARNING: USAspending Archive files are about 10GB per GFY when unzipped (~1GB zipped).")
print()
if not os.path.exists("USAspending_Archive_Downloads"):
    os.mkdir("USAspending_Archive_Downloads")

assert os.path.exists("USAspending_Archive_Downloads")

download_file_path = os.path.join(original_cwd, "USAspending_Archive_Downloads")

df_archive_local_file_info = Get_Info_On_Archive_Files(download_file_path)
df_archive_local_file_info

In [ ]:
# for each file, is there a newer version in the directory? if so, move the older version to old_files_directory
os.chdir(original_cwd)
os.chdir(download_file_path) #!!! changing the working Directory

if not os.path.exists("OLD_FILES_Delete_If_Not_Needed"):
    os.mkdir("OLD_FILES_Delete_If_Not_Needed")

assert os.path.exists("OLD_FILES_Delete_If_Not_Needed")

if not os.path.exists("Expanded_CSV_Files"):
    os.mkdir("Expanded_CSV_Files")

assert os.path.exists("Expanded_CSV_Files")

existing_archive_filenames_zip = sorted(glob.glob(os.path.join(download_file_path, "*.zip")))

existing_archive_filenames_csv = sorted(glob.glob(os.path.join(download_file_path, "Expanded_CSV_Files", "*.csv")))

# test whether there are other files in folder that are not zip nor csv
existing_archive_filenames_other = sorted(set(glob.glob(os.path.join(download_file_path, "*"))) - set(existing_archive_filenames_zip) - set(existing_archive_filenames_csv))
# the only other file should be the directory for the old files for deletion
if existing_archive_filenames_other != [] and len(existing_archive_filenames_other) == 1:
    assert existing_archive_filenames_other[0].split("/")[-1] #confirm this exists - is True
else:
    for filename in existing_archive_filenames_other:
        if not os.path.isdir(filename):
            print(f"unexpected file: {filename} in the download directory. May be ok. Please confirm.")
            print()


def Get_Most_Recent_Archive_Name(gfy_filenames):
    most_recent_file = gfy_filenames[0]
    last_filename_to_check_ArchiveDate = Get_ArchiveDate_from_file_path(most_recent_file)
    files_to_move = []
    for n, filename in enumerate(gfy_filenames[1:]):
        #print(f"{n+1} : {filename}")
        #print(f">> {n+1}, most recent: {Get_Filename_Only(most_recent_file)}, filename: {Get_Filename_Only(filename)}")
        filename_ArchiveDate = Get_ArchiveDate_from_file_path(filename)
        #print(f"{filename_ArchiveDate} > {last_filename_to_check_ArchiveDate}")
        if int(filename_ArchiveDate) > int(last_filename_to_check_ArchiveDate): #True if this is a later file
            last_filename_to_check_ArchiveDate = filename_ArchiveDate
            files_to_move.append(most_recent_file)
            most_recent_file = filename
            #print(f"GFY: {filename_GFY} Files with the same GFY: {filename_to_check_GFY} -> {Get_Filename_Only(filename)} archiveDate: {filename_GFY}")

    file_analysis = {"keep_most_recent_version" :  most_recent_file, "move_older_versions_list" : files_to_move}
    
    return file_analysis

def Identify_Files_to_Move(zip_GFY_dict,existing_archive_filenames_ext):
    results = []
    for gfy, count in zip_GFY_dict.items():
        if count > 1:
            gfy_filenames = sorted([filename for filename in existing_archive_filenames_ext if gfy in filename])
            move_list = Get_Most_Recent_Archive_Name(gfy_filenames)['move_older_versions_list']
            
            # look for improperly formated file names in the .zip and .csv files
            for filename in gfy_filenames:
                if not Check_Archive_Filename_Format(filename):
                    move_list.append(filename)
            
            results.extend(move_list)
    return results

def Move_Files(move_older_versions_list, extension = "*.zip", move_to_folder = "OLD_FILES_Delete_If_Not_Needed"):
    move_to_list = []
    for source_filename in move_older_versions_list:
        source_filename_only = os.path.basename(source_filename)
        source_dir = os.path.dirname(source_filename)
        common_dir = os.path.dirname(source_dir)
        destination_dir = os.path.join(source_dir, move_to_folder) #not moving to two peer directories but from higher dir to lower
        if not os.path.isdir(destination_dir):
            os.mkdir(destination_dir)
        
        move_result = shutil.move(source_filename, destination_dir)
        move_to_list.append(move_result)
    return move_to_list

def Cleanup_Zip_CSV_Directories(download_file_path):
    csv_folder = "Expanded_CSV_Files"
    print(f"Analyzing and cleaning up in your Download Zip and CSV files files...")
    print()
    for extension in ['*.zip', '*.csv']:
        if extension == '*.zip':
            existing_archive_filenames_ext = sorted(glob.glob(os.path.join(download_file_path, extension)))
        if extension == "*.csv":
            existing_archive_filenames_ext = sorted(glob.glob(os.path.join(download_file_path, csv_folder, extension)))
        zip_GFY_dict = dict(Counter([Get_GFY_from_file_path(filename) for filename in existing_archive_filenames_ext]))
        #zip_GFY_dict   
        files_to_move = Identify_Files_to_Move(zip_GFY_dict,existing_archive_filenames_ext)
        moved_files_new_destination = Move_Files(files_to_move , extension, move_to_folder = "OLD_FILES_Delete_If_Not_Needed")
        for source, destination in zip(files_to_move, moved_files_new_destination):
            print(f"Source: {source}")  
            print(f"      -> Destination: {destination}")
    print("Done.")
    return
        
Cleanup_Zip_CSV_Directories(download_file_path)

os.chdir(original_cwd)#!!! confirm working directory is the same as when processing started

### Build a list of URL's to download latest USAspending GFY Archives

In [ ]:
## Key Step - relies on current structure of how USAspending.gov names and locates annual archive files
def Construct_Archive_Download_URL_List():
    #example https://files.usaspending.gov/award_data_archive/FY2019_All_Contracts_Full_20200807.zip
    prefix = "https://files.usaspending.gov/award_data_archive/FY"
    midfix = "_All_Contracts_Full_"
    tail = ".zip"

    latest_archive_extension_date = full_download_filename.split(".")[0].split("_")[-1]

    url_downloads_list = []
    for FY in range(2010,CurrentGFY()+1):
      url_download_name = prefix+str(FY)+midfix+latest_archive_extension_date+tail
      url_downloads_list.append(url_download_name)
    return sorted(url_downloads_list)

url_downloads_list = Construct_Archive_Download_URL_List()

In [ ]:
# decide which files to download using the rule:
#     if the GFY already exists then, only download the file if there is a newer version
#     for the current GFY and the previous GFY on the assumption that curent GFY - 2 is relatively stable (valid assumption?)

os.chdir(download_file_path) # change to the download archive folder
print(f"Files will be downloaded to: {os.getcwd()}")
print()
print("...the download process may take several minutes depending on your internet connections...")

existing_archive_filenames = sorted(glob.glob('*.zip'))

# remove FY's that have been added already except for most recent year GFY2020

existing_GFY_list = sorted([filename.split("/")[-1][:6] for filename in existing_archive_filenames])


#check if the last GFY file archive date has changed since the last download, if so get current GFY and previous GFY
if Get_ArchiveDate_from_file_path(existing_archive_filenames[-1]) != Get_ArchiveDate_from_file_path(url_downloads_list[-1]):
    existing_GFY_list = existing_GFY_list[:-2] # remove current GFY and previous GFY and get them again since that is the current GFY and being updated every 2 weeks

for url_download in url_downloads_list:
    filename = url_download.split("/")[-1]
    GFY = filename[0:6]
    if not GFY in existing_GFY_list: 
        print(f"Downloading: {url_download}")
        !wget $url_download
        print()
    else:
        print(f"Skipping file: {filename} already exists in directory.")

print('Downloads completed.')
print()
zip_archive_size = sum([os.path.getsize(filename) for filename in existing_archive_filenames])/1e9
print(f"USAspending GFY zip file archive size: {zip_archive_size } GB")

os.chdir(original_cwd)

### Clean up the Download Zip and CSV Directory Again

In [ ]:
Cleanup_Zip_CSV_Directories(download_file_path)

os.chdir(original_cwd)#!!! confirm working directory is the same as when processing started

## Begin Processing of Zip Files

In [ ]:
# unzip the files using the zipfile package
# delete the zip files

In [ ]:
select_GFYs_to_Unzip = ['FY2018', 'FY2019', 'FY2020']

zip_filenames_all = sorted(glob.glob(os.path.join(download_file_path, "*.zip")))
for GFY in select_GFYs_to_Unzip:
    
    aZipFileName = [filename for filename in zip_filenames if GFY in filename][0] #get the Zip file of interest to expand

    with zipfile.ZipFile(aZipFileName) as myzip:
        print(f"Extracting files for GFY: {GFY[2:]}...")

        # unzip files to "Expanded_CSV_Files" Directory
        download_file_path = os.path.join(original_cwd, "USAspending_Archive_Downloads")
        download_file_path_CSV = os.path.join(download_file_path, "Expanded_CSV_Files")

        os.chdir(download_file_path) # !!!! change to the download archive folder
        print("Beginning file Unzip Process. This may take a few minutes to process.")
        
        myzip.extractall(path=download_file_path_CSV) # extract them to the directory in path (CSVs only)
        # this should overwrite a previous version of the same name

        os.chdir(original_cwd) #!!! return to the working directory
        print()
        print('Done!')

In [ ]:
# compute size of exanded USAspending csv files
csv_GB = sum([os.path.getsize(filename) for filename in glob.glob(download_file_path_CSV + "/*.csv")])/1e9
print(f"Expanded_CSV-Files folder is {csv_GB} GB")

In [ ]:
# run cleanup again to ensure the files remain consistent for the subsequent steps
Cleanup_Zip_CSV_Directories(download_file_path)

os.chdir(original_cwd)#!!! confirm working directory is the same as when processing started

## Let's Try a Quick Test to Make Sure the files downloaded and expanded properly

In [ ]:
download_file_path_CSV = os.path.join(download_file_path, "Expanded_CSV_Files")
csv_filenames = sorted(glob.glob(download_file_path_CSV + "/*.csv"))
print(f"There are {len(csv_filenames)} *.csv files.")

In [ ]:
# read the first one into a pandas dataframe to confirm that works
print(f"Reading file: {os.path.basename(csv_filenames[0])}")
df_test = pd.read_csv(csv_filenames[0], low_memory=False, nrows=10) #first 10 rows for testing purposes

df_test.head()

In [ ]:
sorted(df_test.columns)

In [ ]:
df_test['federal_action_obligation'].sum() #Sum of Federal Obligations for the first 10 rows of the first CSV file

### Now we are ready to start analyzing the data.

Look for the 4th post in the series with details on processing and analyzing the data.

https://opensource.org/licenses/MIT

MIT Open Source License Copyright 2020 Leif C Ulstrup

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.